In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import os
import openai
import logging
import sys

from llama_index import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
)
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.storage.storage_context import StorageContext


import pinecone

In [68]:
openai.api_key = os.environ["OPENAI_API_KEY"]
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [69]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [70]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

In [71]:
environment = 'gcp-starter'
index_name = "baseline-index"

In [72]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [73]:
# DO NOT RERUN
pinecone.init(api_key=PINECONE_API_KEY)
vector_store = PineconeVectorStore(
    index_name=index_name,
    environment=environment
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

dir_path = 'data'
documents = SimpleDirectoryReader(dir_path).load_data()
vector_index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
)
















Upserted vectors: 100%|██████████| 7/7 [00:01<00:00,  4.37it/s]


In [78]:
vector_index.index_struct.index_id = 'patent'
pinecone_engine = vector_index.as_query_engine()

In [79]:
from helpers import test_review

test_review(pinecone_engine, 'pinecone_review')


INFO:root:Logging to /Users/lynakim/projects/agi-hack-lightbulbml/experiments/sandbox/20231007-1696721031/default.log
Logging to /Users/lynakim/projects/agi-hack-lightbulbml/experiments/sandbox/20231007-1696721031/default.log
Logging to /Users/lynakim/projects/agi-hack-lightbulbml/experiments/sandbox/20231007-1696721031/default.log
